In [33]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta


In [44]:
loco_log=r"c:\Users\hbl\Desktop\automation scripts\rri2csvbot_v1\logscvrt\12348_DN DIRECTION_BIU_SAHID EXP_02.06.25"
#loco_log=r'c:\Users\hbl\Desktop\automation scripts\rri2csvbot_v1\logscvrt\12339_UP _DIRECTION _BIU_COALFIELD EXPRESS _07.07.25'


Data reading

In [45]:
csvfolder=os.path.join(loco_log,"csv_files")
def read_csv_files(csvfolder,file_names):
    dataframes={}
    for file_name in file_names:
        matching_files = [f for f in os.listdir(csvfolder) if file_name in f and f.endswith(".csv")]
        if matching_files:
            file_path= os.path.join(csvfolder,matching_files[0])
            df=pd.read_csv(file_path)
            dataframes[file_name]=df
            print(f"successfully read {file_name}")
        else:
            print(f"failed to read {file_name}")
    return dataframes

file_names = ["LocoRadioRegStringLog", "LocoRadioStringLog", "StnRegLogV4", "StnRespStringLog"]    
csvfiles = read_csv_files(csvfolder, file_names) 
LocoRequest=csvfiles.get("LocoRadioStringLog")
StationResponse=csvfiles.get("StnRespStringLog")
LocoRegular= csvfiles.get("LocoRadioRegStringLog")
StationRegular= csvfiles.get("StnRegLogV4")
LocoRequest=LocoRequest[['FrameNo','TimeStamp','LocoId','Abs Location','Dir','Mode','Appr Stn','Tag-Id']]
StationResponse=StationResponse[['Type', 'Length', 'FrameNo', 'TimeStamp', 'StnId', 'Abs Lctn','Dest LocoId']]
LocoRegular=LocoRegular[['FrameNo', 'TimeStamp', 'LocoId','Abs Location','Dir', 'Emergency', 'Mode', 'Tag-Id','StPrfId']]
StationRegular = StationRegular[['FrameNo', 'TimeStamp', 'Stn-Id', 'DestLocoID', 'LastRefRFID', 'PktDir', 'Frame Offset','TrainSecType']]
loco_id = LocoRegular.loc[LocoRegular['Mode'] == "FS", 'LocoId'].iloc[0] 
print(loco_id)
# getting the loco-idwhere the mode is FS
def data_cleaning(df,loco_id):
    if 'LocoId' in df.columns:
        df=df[df["LocoId"]==loco_id] 
    elif 'DestLocoID' in df.columns:
        df=df[df["DestLocoID"]==loco_id]
    else:
        df=df[df["Dest LocoId"]==loco_id]
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
    df=df.sort_values(by="FrameNo")
    df = df.drop_duplicates(subset=["FrameNo"])
    return df    
LocoRequest=data_cleaning (LocoRequest,loco_id)
StationResponse=data_cleaning (StationResponse,loco_id)
LocoRegular=data_cleaning (LocoRegular,loco_id)
StationRegular =data_cleaning (StationRegular,loco_id)
#print(StationRegular.head(6))


successfully read LocoRadioRegStringLog
successfully read LocoRadioStringLog
successfully read StnRegLogV4
successfully read StnRespStringLog
30115


C:\Users\hbl\AppData\Local\Temp\ipykernel_11756\3419371939.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
C:\Users\hbl\AppData\Local\Temp\ipykernel_11756\3419371939.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
C:\Users\hbl\AppData\Local\Temp\ipykernel_11756\3419371939.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['TimeStamp'] = pd.to_datetime(df['Tim

In [7]:
print(StationRegular.columns)

Index(['FrameNo', 'TimeStamp', 'Stn-Id', 'DestLocoID', 'LastRefRFID', 'PktDir',
       'Frame Offset', 'TrainSecType'],
      dtype='object')


In [8]:
import pandas as pd
import numpy as np

class RFWeakASPBlank:
    def __init__(self, LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction):
        self.direction = direction
        self.loco_id = loco_id
        self.loco_rqst = LocoRequest[LocoRequest['Dir'] == direction]
        self.stn_resp = StationResponse
        self.loco_reg = LocoRegular[LocoRegular['Dir'] == direction]
        self.stn_reg = StationRegular[StationRegular['PktDir'] == direction]
        self.unique_stations = self.loco_rqst['Appr Stn'].unique().tolist()

    def after_registration_check(self, station, reg_df,lreg_time,sreg_time,LSmt):
        nxt_stn_index=self.unique_stations.index(station)
        if nxt_stn_index < len(self.unique_stations) - 1:
            next_station_id = self.unique_stations[nxt_stn_index + 1]
            next_stn_req_time = self.loco_rqst[self.loco_rqst['Appr Stn']==next_station_id]['TimeStamp'].iloc[0]
            Aft_reg_sreg= self.stn_reg[self.stn_reg['Stn-Id']==station].copy()
            last_reg_time=Aft_reg_sreg['TimeStamp'].iloc[-1]
            print(f"curstn={last_reg_time} next stn= {next_stn_req_time} cur-nxt= {(last_reg_time-next_stn_req_time).total_seconds()}")
            if not Aft_reg_sreg.empty:
                Aft_reg_sreg['TimeDifference']=Aft_reg_sreg['TimeStamp'].diff().dt.total_seconds()
                missing=Aft_reg_sreg[Aft_reg_sreg['TimeDifference']>=4]
                
                if missing.empty:
                    print("no stn reg packets were missing")
                elif not LSmt.empty and (LSmt['TimeStamp'] > sreg_time).any():
                    print("LS came after registration")
                    # code for LS mode should be written
                else:
                    print('rf issues came here')
                    if missing.shape[0]>3:
                        if not LSmt.empty and (LSmt['TimeStamp'] > sreg_time).any():
                           print("-> There are intermittent packets missing and LS mode")
                        else:   
                            print("intermittent packets are missing")
                            start,sd = missing['TimeStamp'].iloc[0],missing['TimeDifference'].iloc[0]
                            sd=pd.Timedelta(seconds=sd)
                            start=start-sd
                            print(f"From{start} to {missing['TimeStamp'].iloc[-1]} intermitent station reg packets are missing  ")
                    else:
                        RF_msgs = [
                        f"Station regular packets are missing for {row['TimeDifference']} secs at {row['TimeStamp']}"
                        for _, row in missing.iterrows()
                        ]
                        print(f"=> RF issue came in {station} station due to {', and '.join(RF_msgs)}")

                        
            else:#stnreg is empty
                if not LSmt.empty and (LSmt['TimeStamp'] > sreg_time).any():
                    print("-> There are no stnreg packets and loco went to LS mode")
                else:
                    print("-> After registering there are no stn reg packetsin loco log ")

    def registration_check(self, station):
        try:
            Loco_req_MT= self.loco_rqst[self.loco_rqst['Appr Stn'] == station]
            lrqst_time = Loco_req_MT['TimeStamp'].iloc[0]
            
        except IndexError:
            print(f"No loco request found for station {station}")
            return
        LSmt = Loco_req_MT[(Loco_req_MT['Mode'] == "LS") & (Loco_req_MT['Mode'].shift(1) == "FS")]
        
        
        if not LSmt.empty :
           print(f"loco switched to ls mode in {station} at {LSmt['TimeStamp']} ")
        filtered_resp = self.stn_resp[
            (self.stn_resp['StnId'] == station) & 
            (self.stn_resp['TimeStamp'] >= lrqst_time)
        ]

        sresp_time = lreg_time = sreg_time = np.nan

        if not filtered_resp.empty:
            first_resp_time = filtered_resp['TimeStamp'].iloc[0]
            if first_resp_time - lrqst_time < pd.Timedelta(minutes=5):
                sresp_time = first_resp_time

                lreg_filter = self.loco_reg[
                    (self.loco_reg['TimeStamp'] >= sresp_time) & 
                    (self.loco_reg['StPrfId'] == 0)
                ]
                if not lreg_filter.empty:
                    lreg_time = lreg_filter['TimeStamp'].iloc[0]

                    sreg_filter = self.stn_reg[
                        (self.stn_reg['TimeStamp'] >= lreg_time) & 
                        (self.stn_reg['Stn-Id'] == station)
                    ]
                    if not sreg_filter.empty:
                        sreg_time = sreg_filter['TimeStamp'].iloc[0]

        process_steps = ['Loco request', 'Station response', 'Loco regular', 'Station regular']
        timestamps = [lrqst_time, sresp_time, lreg_time, sreg_time]
        reg_df = pd.DataFrame({'registration process': process_steps, 'time': timestamps})
        reg_df['time difference'] = reg_df['time'].diff().dt.total_seconds()
        print(reg_df)
        stnregtime=reg_df[reg_df['registration process']=="Station regular"]['time'].iloc[0]
        
        #print(stnregtime)
        
        if reg_df['time'].isnull().any() :# LS mode due to loco not registered
            missing = reg_df[reg_df['time'].isnull()]['registration process'].tolist()
            if not LSmt.empty:
                print(f"=> Registration in {station} failed due to missing: {', '.join(missing)} and loco switched to LS mode at {LSmt['TimeStamp']}")
            else:
                print(f"=> Registration in {station} failed due to missing: {', '.join(missing)}")
        elif reg_df['time difference'].sum() <= 6 and not reg_df['time'].isnull().any():
            print(f"=> Loco registration with {station} station was perfect")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)
        elif not LSmt.empty and not reg_df['time'].isnull().any() and LSmt['TimeStamp'].iloc[0]<stnregtime:# ls mode check between registration 
            first_ls_time=LSmt['TimeStamp'].iloc[0]
            delays = reg_df[
                (reg_df['time difference'] > 2) & reg_df['time difference'].notnull()
            ]
            if not delays.empty:
                delay_msgs = [
                    f"{row['registration process']} delayed by {row['time difference']} secs at {row['time']}"
                    for _, row in delays.iterrows()
                ]
                print(f"=> Registration delayed and loco switched to LS mode at {first_ls_time} in {station} station due to {', and '.join(delay_msgs)}")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)
        else:
                
            delays = reg_df[
                (reg_df['time difference'] > 2) & reg_df['time difference'].notnull()
            ]
            if not delays.empty:
                delay_msgs = [
                    f"{row['registration process']} delayed by {row['time difference']} secs at {row['time']}"
                    for _, row in delays.iterrows()
                ]
                print(f"=> Registration delayed in {station} station due to {', and '.join(delay_msgs)}")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)

    def process(self):
        for station in self.unique_stations:
            print(f"\nChecking station: {station}")
            self.registration_check(station)
print("************ Nominal direction analysis ************")
rf_nom = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="N")
rf_nom.process()

print("************ Reverse direction analysis ****************************************************************************************")
rf_rev = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="R")
rf_rev.process()

************ Nominal direction analysis ************
************ Reverse direction analysis ****************************************************************************************

Checking station: 33017
  registration process                time  time difference
0         Loco request 2025-07-29 10:02:09              NaN
1     Station response 2025-07-29 10:02:09              0.0
2         Loco regular 2025-07-29 10:02:11              2.0
3      Station regular 2025-07-29 10:02:15              4.0
=> Loco registration with 33017 station was perfect
curstn=2025-07-29 10:04:15 next stn= 2025-07-29 10:04:13 cur-nxt= 2.0
no stn reg packets were missing

Checking station: 33018
  registration process                time  time difference
0         Loco request 2025-07-29 10:04:13              NaN
1     Station response 2025-07-29 10:04:13              0.0
2         Loco regular 2025-07-29 10:04:17              4.0
3      Station regular 2025-07-29 10:04:19              2.0
=> Loco regist

In [11]:
import pandas as pd
import numpy as np

class RFWeakASPBlank:
    def __init__(self, LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction):
        self.direction = direction
        self.loco_id = loco_id
        self.loco_rqst = LocoRequest[LocoRequest['Dir'] == direction]
        self.stn_resp = StationResponse
        self.loco_reg = LocoRegular[LocoRegular['Dir'] == direction]
        self.stn_reg = StationRegular[StationRegular['PktDir'] == direction]
        self.unique_stations = self.loco_rqst['Appr Stn'].unique().tolist()

    def after_registration_check(self, station, reg_df,lreg_time,sreg_time,LSmt):
        nxt_stn_index=self.unique_stations.index(station)
        if nxt_stn_index < len(self.unique_stations) - 1:
                next_station_id = self.unique_stations[nxt_stn_index + 1]
                next_stn_req_time = self.loco_rqst[self.loco_rqst['Appr Stn'] == next_station_id]['TimeStamp'].iloc[0]
                Aft_reg_sreg = self.stn_reg[self.stn_reg['Stn-Id'] == station].copy()# getting stn reg packets 
                if not Aft_reg_sreg.empty:
                    last_reg_time = Aft_reg_sreg['TimeStamp'].iloc[-1]
                    print(f"curstn={last_reg_time} next stn={next_stn_req_time} cur-nxt={(last_reg_time - next_stn_req_time).total_seconds()}")
                    Aft_reg_sreg['TimeDifference'] = Aft_reg_sreg['TimeStamp'].diff().dt.total_seconds()
                    missing = Aft_reg_sreg[Aft_reg_sreg['TimeDifference'] >= 4]
                    ls_after_reg = not LSmt.empty and (LSmt['TimeStamp'] > sreg_time).any()

                    if missing.empty and not ls_after_reg:# no RF and LS issue
                        print(f"->No station regular packets are missing in{station}")
                    else:#missing not empty
                        if not ls_after_reg.empty:# station packets are missing and LS mode came
                            print("=-> LS mode was received after registration")
                
                        if not missing.empty:# only RF issue found 
                            if missing.shape[0] > 3: # more than 2 rf issues are found
                                start_time = missing['TimeStamp'].iloc[0] - pd.Timedelta(seconds=missing['TimeDifference'].iloc[0])
                                end_time = missing['TimeStamp'].iloc[-1]
                                print(f"-> In {station} station from {start_time} to {end_time}, intermittent station registration packets are missing.")
                            else:# less than 3 rf issue but only RF issues
                                rf_msgs = [
                                    f"{row['TimeDifference']} secs at {row['TimeStamp']}"
                                    for _, row in missing.iterrows()
                                ]
                                print(f"=> RF issue in {station} due to {', and '.join(rf_msgs)}")
                        if ls_after_reg and not missing.empty:# both Rf and LS issues found together with stn reg missing
                            print("->Note: Both RF anomalies and LS mode transition observed.")
                else:
                    ls_after_reg = not LSmt.empty and (LSmt['TimeStamp'] > sreg_time).any()
                    if ls_after_reg:# no stn reg packets and LS received
                        print("-> No station registration packets found; loco switched to LS mode.")
                    else:#no LS and no stnr
                         print("-> No station registration packets present in loco log after registration and LS mode is not received.")



            

    def registration_check(self, station):
        try:
            Loco_req_MT= self.loco_rqst[self.loco_rqst['Appr Stn'] == station]
            lrqst_time = Loco_req_MT['TimeStamp'].iloc[0]
            
        except IndexError:
            print(f"No loco request found for station {station}")
            return
        LSmt = Loco_req_MT[(Loco_req_MT['Mode'] == "LS") & (Loco_req_MT['Mode'].shift(1) == "FS")]
        
        
        if not LSmt.empty :
           print(f"loco switched to ls mode in {station} at {LSmt['TimeStamp']} ")
        filtered_resp = self.stn_resp[
            (self.stn_resp['StnId'] == station) & 
            (self.stn_resp['TimeStamp'] >= lrqst_time)
        ]

        sresp_time = lreg_time = sreg_time = np.nan

        if not filtered_resp.empty:
            first_resp_time = filtered_resp['TimeStamp'].iloc[0]
            if first_resp_time - lrqst_time < pd.Timedelta(minutes=5):
                sresp_time = first_resp_time

                lreg_filter = self.loco_reg[
                    (self.loco_reg['TimeStamp'] >= sresp_time) & 
                    (self.loco_reg['StPrfId'] == 0)
                ]
                if not lreg_filter.empty:
                    lreg_time = lreg_filter['TimeStamp'].iloc[0]

                    sreg_filter = self.stn_reg[
                        (self.stn_reg['TimeStamp'] >= lreg_time) & 
                        (self.stn_reg['Stn-Id'] == station)
                    ]
                    if not sreg_filter.empty:
                        sreg_time = sreg_filter['TimeStamp'].iloc[0]

        process_steps = ['Loco request', 'Station response', 'Loco regular', 'Station regular']
        timestamps = [lrqst_time, sresp_time, lreg_time, sreg_time]
        reg_df = pd.DataFrame({'registration process': process_steps, 'time': timestamps})
        reg_df['time difference'] = reg_df['time'].diff().dt.total_seconds()
        print(reg_df)
        stnregtime=reg_df[reg_df['registration process']=="Station regular"]['time'].iloc[0]
        
        #print(stnregtime)
        
        if reg_df['time'].isnull().any() :# LS mode due to loco not registered
            missing = reg_df[reg_df['time'].isnull()]['registration process'].tolist()
            if not LSmt.empty:
                print(f"=> Registration in {station} failed due to missing: {', '.join(missing)} and loco switched to LS mode at {LSmt['TimeStamp']}")
            else:
                print(f"=> Registration in {station} failed due to missing: {', '.join(missing)}")
        elif reg_df['time difference'].sum() <= 6 and not reg_df['time'].isnull().any():
            print(f"=> Loco registration with {station} station was perfect")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)
        elif not LSmt.empty and not reg_df['time'].isnull().any() and (LSmt['TimeStamp']<stnregtime).any() :# ls mode check between registration 
            first_ls_time=LSmt['TimeStamp'].iloc[0]
            delays = reg_df[
                (reg_df['time difference'] > 2) & reg_df['time difference'].notnull()
            ]
            if not delays.empty:
                delay_msgs = [
                    f"{row['registration process']} delayed by {row['time difference']} secs at {row['time']}"
                    for _, row in delays.iterrows()
                ]
                print(f"=> Registration delayed and loco switched to LS mode at {first_ls_time} in {station} station due to {', and '.join(delay_msgs)}")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)
        else:
                
            delays = reg_df[
                (reg_df['time difference'] > 2) & reg_df['time difference'].notnull()
            ]
            if not delays.empty:
                delay_msgs = [
                    f"{row['registration process']} delayed by {row['time difference']} secs at {row['time']}"
                    for _, row in delays.iterrows()
                ]
                print(f"=> Registration delayed in {station} station due to {', and '.join(delay_msgs)}")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)

    def process(self):
        for station in self.unique_stations:
            print(f"\nChecking station: {station}")
            self.registration_check(station)
print("************ Nominal direction analysis ************")
rf_nom = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="N")
rf_nom.process()

print("************ Reverse direction analysis ****************************************************************************************")
rf_rev = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="R")
rf_rev.process()

************ Nominal direction analysis ************
************ Reverse direction analysis ****************************************************************************************

Checking station: 33017
  registration process                time  time difference
0         Loco request 2025-07-29 10:02:09              NaN
1     Station response 2025-07-29 10:02:09              0.0
2         Loco regular 2025-07-29 10:02:11              2.0
3      Station regular 2025-07-29 10:02:15              4.0
=> Loco registration with 33017 station was perfect
curstn=2025-07-29 10:04:15 next stn=2025-07-29 10:04:13 cur-nxt=2.0
->No station regular packets are missing in33017

Checking station: 33018
  registration process                time  time difference
0         Loco request 2025-07-29 10:04:13              NaN
1     Station response 2025-07-29 10:04:13              0.0
2         Loco regular 2025-07-29 10:04:17              4.0
3      Station regular 2025-07-29 10:04:19              2.0

AttributeError: 'bool' object has no attribute 'empty'

In [ ]:
import pandas as pd
import numpy as np

class RFWeakASPBlank:
    def __init__(self, LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction):
        self.direction = direction
        self.loco_id = loco_id
        self.loco_rqst = LocoRequest[LocoRequest['Dir'] == direction]
        self.stn_resp = StationResponse
        self.loco_reg = LocoRegular[LocoRegular['Dir'] == direction]
        self.stn_reg = StationRegular[StationRegular['PktDir'] == direction]
        self.unique_stations = self.loco_rqst['Appr Stn'].unique().tolist()

    def after_registration_check(self, station, reg_df,lreg_time,sreg_time,LSmt):
        Aft_reg_sreg = self.stn_reg[self.stn_reg['Stn-Id'] == station].copy()# getting stn reg packets 
        if not Aft_reg_sreg.empty:
            last_reg_time = Aft_reg_sreg['TimeStamp'].iloc[-1]
            Aft_reg_sreg['TimeDifference'] = Aft_reg_sreg['TimeStamp'].diff().dt.total_seconds()
           # missing = Aft_reg_sreg[Aft_reg_sreg['TimeDifference'] >= 4]
            ls_after_reg = LSmt[LSmt['TimeStamp'] > sreg_time]
            if not ls_after_reg.empty:
                #print(ls_after_reg)
                for index,rows in ls_after_reg.iterrows():
                    ls_time_aft_reg=rows['TimeStamp']
                    stn_reg_ls=Aft_reg_sreg[Aft_reg_sreg['TimeStamp']<=ls_time_aft_reg]
                    stn_reg_ls=stn_reg_ls.tail(1)
                    if stn_reg_ls['Frame Offset'].iloc[0]>3:#stn reg missing causing LS with out frame offset
                        loco_reg_ls=self.stn_resp[self.stn_resp['TimeStamp']<ls_time_aft_reg].copy
                        print(f"loco went to ls mode due to frame off set {stn_reg_ls['Frame Offset'].iloc[0]} ")
                        
                    else:
                        starttime=stn_reg_ls['TimeStamp'].iloc[0]
                        timedif=ls_time_aft_reg-starttime
                        print(f"From {starttime} to {ls_time_aft_reg} station regular packets are missing for {timedif} secs ")
                            
            else:
                print(f"LS mode not received in {station}")

        else:
            ls_after_reg = not LSmt.empty and (LSmt['TimeStamp'] > sreg_time).any()
            if ls_after_reg:# no stn reg packets and LS received
                print("-> No station registration packets found; loco switched to LS mode.")
            else:#no LS and no stnr
                 print("-> No station registration packets present in loco log after registration and LS mode is also not received.")



            

    def registration_check(self, station):
        try:
            Loco_req_MT= self.loco_rqst[self.loco_rqst['Appr Stn'] == station]
            lrqst_time = Loco_req_MT['TimeStamp'].iloc[0]
            
        except IndexError:
            print(f"No loco request found for station {station}")
            return
        LSmt = Loco_req_MT[(Loco_req_MT['Mode'] == "LS") & (Loco_req_MT['Mode'].shift(1) == "FS")]
        
        
        if not LSmt.empty :
           print(f"loco switched to ls mode in {station} at {LSmt['TimeStamp']} ")
           print(LSmt)
        filtered_resp = self.stn_resp[
            (self.stn_resp['StnId'] == station) & 
            (self.stn_resp['TimeStamp'] >= lrqst_time)
        ]

        sresp_time = lreg_time = sreg_time = np.nan

        if not filtered_resp.empty:
            first_resp_time = filtered_resp['TimeStamp'].iloc[0]
            if first_resp_time - lrqst_time < pd.Timedelta(minutes=5):
                sresp_time = first_resp_time

                lreg_filter = self.loco_reg[
                    (self.loco_reg['TimeStamp'] >= sresp_time) & 
                    (self.loco_reg['StPrfId'] == 0)
                ]
                if not lreg_filter.empty:
                    lreg_time = lreg_filter['TimeStamp'].iloc[0]

                    sreg_filter = self.stn_reg[
                        (self.stn_reg['TimeStamp'] >= lreg_time) & 
                        (self.stn_reg['Stn-Id'] == station)
                    ]
                    if not sreg_filter.empty:
                        sreg_time = sreg_filter['TimeStamp'].iloc[0]

        process_steps = ['Loco request', 'Station response', 'Loco regular', 'Station regular']
        timestamps = [lrqst_time, sresp_time, lreg_time, sreg_time]
        reg_df = pd.DataFrame({'registration process': process_steps, 'time': timestamps})
        reg_df['time difference'] = reg_df['time'].diff().dt.total_seconds()
        #print(reg_df)
        stnregtime=reg_df[reg_df['registration process']=="Station regular"]['time'].iloc[0]
        
        #print(stnregtime)
        
        if reg_df['time'].isnull().any() :# LS mode due to loco not registered
            missing = reg_df[reg_df['time'].isnull()]['registration process'].tolist()
            if not LSmt.empty:
                print(f"=> Registration in {station} failed due to missing: {', '.join(missing)} and loco switched to LS mode at {LSmt['TimeStamp']}")
            else:
                print(f"=> Registration in {station} failed due to missing: {', '.join(missing)}")
        elif reg_df['time difference'].sum() <= 6 and not reg_df['time'].isnull().any():
            print(f"=> Loco registration with {station} station was perfect")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)
        elif not LSmt.empty and not reg_df['time'].isnull().any() and (LSmt['TimeStamp']<stnregtime).any() :# ls mode check between registration 
            first_ls_time=LSmt['TimeStamp'].iloc[0]
            delays = reg_df[
                (reg_df['time difference'] > 2) & reg_df['time difference'].notnull()
            ]
            if not delays.empty:
                delay_msgs = [
                    f"{row['registration process']} delayed by {row['time difference']} secs at {row['time']}"
                    for _, row in delays.iterrows()
                ]
                print(f"=> Registration delayed and loco switched to LS mode at {first_ls_time} in {station} station due to {', and '.join(delay_msgs)}")
            else:
                print("LS mode might be caused by earlier deregistration with previous station")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)
        else:
                
            delays = reg_df[
                (reg_df['time difference'] > 2) & reg_df['time difference'].notnull()
            ]
            if not delays.empty:
                delay_msgs = [
                    f"{row['registration process']} delayed by {row['time difference']} secs at {row['time']}"
                    for _, row in delays.iterrows()
                ]
                print(f"=> Registration delayed in {station} station due to {', and '.join(delay_msgs)}")
            self.after_registration_check(station, reg_df,lreg_time,sreg_time,LSmt)

    def process(self):
        for station in self.unique_stations:
            print(f"\nChecking station: {station}")
            self.registration_check(station)
print("************ Nominal direction analysis ************")
rf_nom = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="N")
rf_nom.process()

print("************ Reverse direction analysis ****************************************************************************************")
rf_rev = RFWeakASPBlank(LocoRequest, StationResponse, LocoRegular, StationRegular, loco_id, direction="R")
rf_rev.process()

************ Nominal direction analysis ************

Checking station: 12000
=> Registration in 12000 failed due to missing: Station response, Loco regular, Station regular

Checking station: 12001
=> Loco registration with 12001 station was perfect
LS mode not received in {station}

Checking station: 12002
=> Registration delayed in 12002 station due to Loco regular delayed by 4.0 secs at 2025-07-29 12:27:09
LS mode not received in {station}

Checking station: 12003
=> Loco registration with 12003 station was perfect
LS mode not received in {station}

Checking station: 12004
=> Registration delayed in 12004 station due to Loco regular delayed by 4.0 secs at 2025-07-29 12:36:41
LS mode not received in {station}

Checking station: 12006
=> Registration delayed in 12006 station due to Station regular delayed by 4.0 secs at 2025-07-29 12:39:11
LS mode not received in {station}

Checking station: 12007
=> Registration delayed in 12007 station due to Station regular delayed by 4.0 secs at 